In [1]:
# Install all required packages
!pip install transformers torch accelerate bitsandbytes peft trl sentence-transformers langchain chromadb rank_bm25 PyMuPDF pandas datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 107.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 113.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.6 MB/s e

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import fitz  # PyMuPDF
import re
import os

# Create a directory for data
os.makedirs('data', exist_ok=True)

def extract_text_from_pdf(pdf_path):
    """Extracts raw text from a PDF file."""
    print(f"Extracting text from {pdf_path}...")
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def clean_text(text):
    """Cleans the text by removing noise and normalizing whitespace."""
    print("Cleaning text...")
    text = re.sub(r'Page \d+ of \d+', '', text) # Remove page numbers
    text = re.sub(r'\s*\n\s*', '\n', text) # Normalize whitespace
    text = re.sub(r'Microsoft Corporation\s+Form 10-K', '', text, flags=re.IGNORECASE) # Remove common headers
    return text.strip()

# --- Main Execution ---
# IMPORTANT: Make sure you have uploaded your PDFs
pdf_path_2023 = '/content/drive/My Drive/data/MSFT_2023_10K.pdf'
pdf_path_2022 = '/content/drive/My Drive/data/MSFT_2022_10K.pdf'

msft_2023_raw = extract_text_from_pdf(pdf_path_2023)
msft_2022_raw = extract_text_from_pdf(pdf_path_2022)

msft_2023_clean = clean_text(msft_2023_raw)
msft_2022_clean = clean_text(msft_2022_raw)

full_text = "--- 2023 Report ---\n" + msft_2023_clean + "\n\n--- 2022 Report ---\n" + msft_2022_clean

processed_text_path = 'data/processed_financials.txt'
with open(processed_text_path, 'w', encoding='utf-8') as f:
    f.write(full_text)

print(f"Data preprocessing complete. Cleaned text saved to '{processed_text_path}'")

Extracting text from /content/drive/My Drive/data/MSFT_2023_10K.pdf...
Extracting text from /content/drive/My Drive/data/MSFT_2022_10K.pdf...
Cleaning text...
Cleaning text...
Data preprocessing complete. Cleaned text saved to 'data/processed_financials.txt'


In [ ]:
import pandas as pd

# Create a dummy CSV for demonstration if you haven't uploaded one
try:
    qa_df = pd.read_csv('qa_dataset.csv')
    print("Q&A Dataset loaded successfully!")
except FileNotFoundError:
    print("Creating a dummy 'qa_dataset.csv'. Please upload your own for real results.")
    dummy_data = {
        'question': ["What was Microsoft's total revenue in fiscal year 2023?", "How much did Microsoft spend on research and development in 2023?"],
        'answer': ["In fiscal year 2023, Microsoft's total revenue was $211.9 billion.", "Microsoft spent $27.2 billion on research and development in 2023."]
    }
    qa_df = pd.DataFrame(dummy_data)
    qa_df.to_csv('qa_dataset.csv', index=False)

print(qa_df.head())

Q&A Dataset loaded successfully!
                                            question  \
0  What was Microsoft's total revenue for the fis...   
1        What was the total cost of revenue in 2023?   
2   What was the gross margin for Microsoft in 2023?   
3  How much did Microsoft spend on Research and D...   
4  What was the total for Sales and marketing exp...   

                                              answer  
0  Microsoft's total revenue for the fiscal year ...  
1  The total cost of revenue in 2023 was $65.9 bi...  
2  Microsoft's gross margin in 2023 was $146.0 bi...  
3  In 2023, Microsoft spent $27.2 billion on Rese...  
4  Sales and marketing expenses totaled $22.7 bil...  


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import chromadb
from rank_bm25 import BM25Okapi
import numpy as np

with open('data/processed_financials.txt', 'r', encoding='utf-8') as f:
    full_text = f.read()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
chunks = text_splitter.split_text(full_text)
print(f"Created {len(chunks)} text chunks.")

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Build Dense Vector Store (ChromaDB)
client = chromadb.Client()
collection = client.get_or_create_collection(name="financials_rag_small")
chunk_embeddings = embedding_model.encode(chunks, show_progress_bar=True)
collection.add(
    ids=[str(i) for i in range(len(chunks))],
    embeddings=chunk_embeddings.tolist(),
    documents=chunks
)
print("Dense vector store created.")

# Build Sparse Index (BM25)
tokenized_chunks = [chunk.lower().split() for chunk in chunks]
bm25 = BM25Okapi(tokenized_chunks)
print("Sparse index (BM25) created.")

Created 2174 text chunks.


Batches:   0%|          | 0/68 [00:00<?, ?it/s]

Dense vector store created.
Sparse index (BM25) created.


In [ ]:
def hybrid_retrieval(query, top_k=5):
    """Performs hybrid retrieval using BM25 and vector search, fused with RRF."""
    processed_query = query.lower()

    # Dense Retrieval
    query_embedding = embedding_model.encode(processed_query).tolist()
    dense_results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    dense_docs = dense_results['documents'][0]

    # Sparse Retrieval
    tokenized_query = processed_query.split()
    bm25_scores = bm25.get_scores(tokenized_query)
    top_n_indices = np.argsort(bm25_scores)[::-1][:top_k]
    sparse_docs = [chunks[i] for i in top_n_indices]

    # RRF Fusion
    fused_scores = {}
    k = 60
    for i, doc in enumerate(dense_docs):
        fused_scores[doc] = fused_scores.get(doc, 0) + 1 / (k + i + 1)
    for i, doc in enumerate(sparse_docs):
        fused_scores[doc] = fused_scores.get(doc, 0) + 1 / (k + i + 1)

    reranked_results = sorted(fused_scores.items(), key=lambda item: item[1], reverse=True)
    return [doc for doc, score in reranked_results][:top_k]

# Test the retriever
test_query = "What was the revenue in 2023?"
retrieved_context = hybrid_retrieval(test_query)
print(f"--- Retrieved Context for '{test_query}' ---")
# Fix: Move slicing outside the f-string
context_snippet = '\n\n'.join(retrieved_context)[:1000]
print(f"{context_snippet}...")

--- Retrieved Context for 'What was the revenue in 2023?' ---
in future periods, was $229 billion as of June 30, 2023, of which $224 billion is related to the commercial portion of revenue. We expect to recognize
approximately 45% of this revenue over the next 12 months and the remainder thereafter.
NOTE 14 — LEASES

determining revenue recognition for these customer agreements was extensive and required a high degree of auditor judgment.
How the Critical Audit Matter Was Addressed in the Audit
Our principal audit procedures related to the Company's revenue recognition for these customer agreements included the following:
•

10-K) for further discussion.
The following table outlines the expected future recognition of unearned revenue as of June 30, 2023:
(In millions)
Three Months Ending
September 30, 2023
$
19,673
December 31, 2023
15,600
March 31, 2024
10,801
June 30, 2024
4,827
Thereafter
2,912
Total
$
53,813

recognition for these customer agreements was extensive and required a hi

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import time

# Load the small model and tokenizer
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Add a pad token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32, # Use standard precision
    device_map="auto",
)
print("Base model DistilGPT-2 loaded successfully.")

Base model DistilGPT-2 loaded successfully.


In [ ]:
def generate_answer(query, context, model_to_use):
    """Generates an answer using a model, context, and query."""
    # Simpler prompt for a base model
    prompt = f"""
    Context:
    {context}

    Question:
    {query}

    Answer:
    """
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True).to("cpu") # Change to "cpu"

    start_time = time.time()
    outputs = model_to_use.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.1,
        pad_token_id=tokenizer.pad_token_id,
        top_k=10
    )
    end_time = time.time()

    # Decode only the newly generated tokens
    answer = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True).strip()
    response_time = end_time - start_time
    return answer, response_time

def rag_output_guardrail(answer):
    """Flags answers that are non-committal."""
    low_confidence_phrases = ["not mentioned", "not available", "i cannot answer"]
    is_confident = not any(phrase in answer.lower() for phrase in low_confidence_phrases)
    confidence_score = 0.90 if is_confident else 0.40
    return answer, confidence_score

# --- Full RAG Pipeline Test ---
def answer_query_rag(query):
    retrieved_chunks = hybrid_retrieval(query)
    context = "\n\n".join(retrieved_chunks)
    answer, response_time = generate_answer(query, context, base_model)
    final_answer, confidence = rag_output_guardrail(answer)
    return final_answer, confidence, response_time

# Example run
test_query = "What was Microsoft's revenue in fiscal year 2023?"
answer, conf, r_time = answer_query_rag(test_query)
print(f"RAG Answer: {answer}")
print(f"Confidence: {conf}")
print(f"Response Time: {r_time:.2f}s")

The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


RAG Answer: What was Microsoft's revenue in fiscal year 2023 compared with fiscal year 2022 included:
•Microsoft Cloud (formerly commercial cloud) revenue increased 32% to $91.2 billion.
•Office Commercial products and cloud services revenue increased 13% driven by Office 365 Commercial growth of 18%.
•Office Consumer products and cloud services revenue increased 13% driven by Office 365 Commercial growth of 18%.
•Office Consumer products and cloud services revenue increased 13% driven by Office 365 Commercial growth
Confidence: 0.9
Response Time: 4.99s


In [ ]:
from datasets import Dataset

def create_raft_dataset(qa_df, chunks):
    raft_data = []
    print("Creating RAFT dataset...")
    for index, row in qa_df.iterrows():
        question, answer = row['question'], row['answer']
        context = "\n\n".join(hybrid_retrieval(question, top_k=1))
        if context:
            # Format for fine-tuning with the simpler prompt
            text = f"""
            Context:
            {context}

            Question:
            {question}

            Answer:
            {answer}
            """
            raft_data.append({"text": text})
    return Dataset.from_list(raft_data)

raft_dataset = create_raft_dataset(qa_df, chunks)
print("RAFT dataset created successfully.")
print(raft_dataset[0]['text'])

Creating RAFT dataset...
RAFT dataset created successfully.

            Context:
            Highlights from fiscal year 2023 compared with fiscal year 2022 included:
•Microsoft Cloud revenue increased 22% to $111.6 billion.
•Office Commercial products and cloud services revenue increased 10% driven by Office 365 Commercial growth of 13%.
•Office Consumer products and cloud services revenue increased 2% and Microsoft 365 Consumer subscribers increased to 67.0 million.

            Question:
            What was Microsoft's total revenue for the fiscal year ended June 30, 2023?

            Answer:
            Microsoft's total revenue for the fiscal year ended June 30, 2023, was $211.9 billion.
            


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
from transformers import TrainingArguments

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["c_attn"], # Target attention layers in GPT-2
    bias="none",
    task_type="CAUSAL_LM",
)

ft_model = get_peft_model(base_model, lora_config)
ft_model.print_trainable_parameters()

training_args = TrainingArguments(
    output_dir="./ft_model_distilgpt2",
    per_device_train_batch_size=4, # Can use a larger batch size
    gradient_accumulation_steps=2,
    learning_rate=5e-5, # A common learning rate for smaller models
    num_train_epochs=10,
    logging_steps=5,
    fp16=True,
    save_strategy="epoch",
)

trainer = SFTTrainer(
    model=ft_model,
    train_dataset=raft_dataset,  # Pass the text data directly
    args=training_args,
)

print("--- Starting Fine-Tuning ---")
trainer.train()
print("--- Fine-Tuning Complete ---")

trainer.save_model("./ft_model_distilgpt2_adapters")

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


trainable params: 147,456 || all params: 82,060,032 || trainable%: 0.1797


Adding EOS to train dataset:   0%|          | 0/101 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/101 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/101 [00:00<?, ? examples/s]

--- Starting Fine-Tuning ---


Step,Training Loss
5,2.639000
10,2.603300
15,2.601700
20,2.649200
25,2.644000
30,2.555100
35,2.630000
40,2.542700
45,2.603700
50,2.557000


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

--- Fine-Tuning Complete ---


In [ ]:
from peft import PeftModel

print("Loading fine-tuned model...")
# Reload the base model to avoid memory issues
base_model_for_ft = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
ft_model_merged = PeftModel.from_pretrained(base_model_for_ft, "./ft_model_distilgpt2_adapters")
ft_model_merged = ft_model_merged.merge_and_unload()
print("Fine-tuned model loaded and merged.")

def answer_query_ft(query):
    context = "\n\n".join(hybrid_retrieval(query, top_k=1))
    answer, response_time = generate_answer(query, context, ft_model_merged)
    final_answer, confidence = rag_output_guardrail(answer)
    return final_answer, confidence, response_time

# --- Run your evaluation questions ---
evaluation_questions = [
    "What was Microsoft's revenue in 2023?",
    "What are the primary strategic risks related to AI development?",
    "What is the capital of France?",
    "How much cash was used in financing activities in 2023?",
    "Compare the net income of 2023 and 2022.",
]

results = []
print("\n--- Starting Evaluation ---")
for q in evaluation_questions:
    print(f"Processing question: {q}")
    rag_ans, rag_conf, rag_time = answer_query_rag(q)
    results.append(["RAG (DistilGPT2)", q, rag_ans, rag_conf, rag_time])

    ft_ans, ft_conf, ft_time = answer_query_ft(q)
    results.append(["Fine-Tune (DistilGPT2)", q, ft_ans, ft_conf, ft_time])

results_df = pd.DataFrame(results, columns=["Method", "Question", "Answer", "Confidence", "Time (s)"])
print("\n--- Evaluation Results ---")
print(results_df)

results_df.to_csv("evaluation_results_small_model.csv", index=False)

Loading fine-tuned model...


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:2156: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Fine-tuned model loaded and merged.

--- Starting Evaluation ---
Processing question: What was Microsoft's revenue in 2023?


The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Processing question: What are the primary strategic risks related to AI development?


...

Processing question: How much cash was used in financing activities in 2023?


The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Processing question: Compare the net income of 2023 and 2022.


The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- Evaluation Results ---
                   Method                                           Question  \
0        RAG (DistilGPT2)              What was Microsoft's revenue in 2023?   
1  Fine-Tune (DistilGPT2)              What was Microsoft's revenue in 2023?   
2        RAG (DistilGPT2)  What are the primary strategic risks related t...   
3  Fine-Tune (DistilGPT2)  What are the primary strategic risks related t...   
4        RAG (DistilGPT2)                     What is the capital of France?   
5  Fine-Tune (DistilGPT2)                     What is the capital of France?   
6        RAG (DistilGPT2)  How much cash was used in financing activities...   
7  Fine-Tune (DistilGPT2)  How much cash was used in financing activities...   
8        RAG (DistilGPT2)           Compare the net income of 2023 and 2022.   
9  Fine-Tune (DistilGPT2)           Compare the net income of 2023 and 2022.   

                                              Answer  Confidence  Time (s)  
0  The company